# Importação de bibliotecas

In [18]:
import pandas as pd
import random
import locale
import os
import openai
import datetime
import re

# Declaração de variáveis

In [19]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

openai.api_key = os.getenv('OPENAI_API_KEY')

NUM_CARTAS_TOTAIS = 78
NUM_CARTAS_ESCOLHIDAS = 5
ARQUIVO_CARTAS = "C:/Users/PC/Desktop/Projetos/Tarot_Python/TAROT - TODAS.xlsx"

# Funções

In [20]:
def escolher_ids_aleatorios():
    return random.sample(range(1, NUM_CARTAS_TOTAIS + 1), NUM_CARTAS_ESCOLHIDAS)


In [21]:
def ler_arquivo(ids_escolhidos):
    df = pd.read_excel(ARQUIVO_CARTAS)
    cartas_selecionadas = df[df['ID'].isin(ids_escolhidos)]
    return cartas_selecionadas


In [22]:
def mostrar_cartas_selecionadas(cartas, conselho):
    for _, carta in cartas.iterrows():
        print(f"Nome: {carta['Carta']}")  
        print(f"Elemento: {carta['Elemento']}")
        print(f"Planeta: {carta['Planeta']}")
        print(f"Signo Astrológico: {carta['Signo_Astrologico']}")
        print(f"Kabbalah: {carta['Cabala_Kabbalah']}")
        print(f"Sim ou Não: {carta['Sim_Ou_Nao']}")
        print(f"Significado: {carta['Significado_Normal']}\n\n")
    print(f"Conselho: {conselho}\n\n")


In [23]:
def exibir_menu_e_capturar_escolha():
    opcao = None
    # Temas para conselhos genéricos
    temas = {
        1: "Amor",
        2: "Dinheiro",
        3: "Amigos",
        4: "Carreira",
        5: "Estudos",
        6: "Espiritualidade",
        7: "Digite sua própria pergunta"
    }
    
    print("Pense em uma pergunta sobre os temas ou digite sua própria pergunta:")
    for chave, valor in temas.items():
        print(f"{chave} - {valor}")
    print("0 - Para sair")

    try:
        opcao = int(input("Escolha uma opção ou digite '7' para personalizar: "))
        if opcao == 0:
            return 0, "Sair"
        elif opcao == 7:
            pergunta_personalizada = input("Digite sua pergunta: ")
            return opcao, pergunta_personalizada
        elif opcao in temas:
            return opcao, temas[opcao]
        else:
            print("Opção inválida. Por favor, digite um número válido.")
            return None, None
    except ValueError:
        print("Por favor, digite um número válido.")
        return None, None


In [30]:
def tema_sem_carac_esp(tema):
    caracteres_invalidos = r'[<>:"/\\|?*]'
    tema_limpo = re.sub(caracteres_invalidos, '_', tema)
    return tema_limpo

In [31]:
def gerar_conselho(tema, cartas_selecionadas):
    # Preparar o prompt com base nas cartas selecionadas e o tema
    cartas_info = ""
    for _, carta in cartas_selecionadas.iterrows():
        cartas_info += f"{carta['Carta']}: {carta['Significado_Normal']} ({'Sim' if carta['Sim_Ou_Nao'] == 'Sim' else 'Não'}).\n"

    prompt = f"Dê um conselho curto (máximo de 280 caracteres) sobre o tema {tema}, considerando as cartas: {', '.join(cartas_selecionadas['Carta'])}. Informações das cartas: {cartas_info}"

    # Escrever o nome do arquivo + tema + data + hora
    agora = datetime.datetime.now()
    tema_limpo = tema_sem_carac_esp(tema)
    data_hora_formatada = agora.strftime("%Y-%m-%d_%H-%M-%S")
    nome_arquivo = f"conselho_{tema_limpo}_{data_hora_formatada}.txt"

    conselho = ''  # Inicializa a variável conselho
    try:
        # Chama a API
        resposta = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "Responda em português: Você é um tarólogo dando conselhos,..."},
                {"role": "user", "content": prompt + " " + cartas_info}
            ]
        )
        conselho = resposta.choices[0].message['content'].strip()
    except Exception as e:
        conselho = f"Não foi possível gerar um conselho devido ao seguinte erro: {e}"

    # Salvar o conselho em um arquivo
    try:
        with open(nome_arquivo, "w", encoding="utf-8") as arquivo:
            arquivo.write(conselho)
    except Exception as e:
        print(f"Não foi possível salvar o arquivo devido ao seguinte erro: {e}")

    return conselho


In [26]:
def main():
    while True:
        # Chama o Menu
        opcao, tema_ou_pergunta = exibir_menu_e_capturar_escolha()

        # 0 para sair
        if opcao == 0:
            print("Volte sempre!")
            break
        # Outros assuntos ou perguntas
        elif opcao in [1, 2, 3, 4, 5, 6, 7]:  
            #Sorteia as cartas aleatoriamente
            ids_escolhidos = escolher_ids_aleatorios()
            # Pega as informações no arquivo
            cartas_escolhidas = ler_arquivo(ids_escolhidos)
            #Pergunta
            if opcao == 7:
                conselho = gerar_conselho(tema_ou_pergunta, cartas_escolhidas)  
                print("Sua pergunta foi:", tema_ou_pergunta)  
            #Conselho generico
            else:
                conselho = gerar_conselho(tema_ou_pergunta, cartas_escolhidas)  
                print("O tema escolhido foi:", tema_ou_pergunta) 
            # Resultado
            mostrar_cartas_selecionadas(cartas_escolhidas, conselho)
        else:
            print('Opção inválida. Tente novamente.')  

In [32]:
main()

Pense em uma pergunta sobre os temas ou digite sua própria pergunta:
1 - Amor
2 - Dinheiro
3 - Amigos
4 - Carreira
5 - Estudos
6 - Espiritualidade
7 - Digite sua própria pergunta
0 - Para sair
Sua pergunta foi: Serei promovida?
Nome: O Papa
Elemento: Terra
Planeta: Venus
Signo Astrológico: Touro
Kabbalah: 16º Chesed-Chokmah
Sim ou Não: Talvez
Significado: autoridade, convencionalidade, conformismo, a importancia da espiritualidade, educacao


Nome: 8 de Copas
Elemento: Agua
Planeta: Saturno
Signo Astrológico: Peixes
Kabbalah: Meio do inverno
Sim ou Não: Nao
Significado: abandono, deixar ir, viajar, escapismo, introspeccao, auto descoberta, solidao.


Nome: Valete de Paus
Elemento: Fogo
Planeta: Jupiter
Signo Astrológico: Sagitario
Kabbalah: Primavera
Sim ou Não: Sim
Significado: inspiracao, aventuras, ideias, criatividade, alegria, potencial ilimitado.


Nome: As de Espada
Elemento: Ar
Planeta: Mercurio e Saturno
Signo Astrológico: Gemeos, Libra e Aquario
Kabbalah: Vau= Ar = Yetzirah =